# PCA for Satellite Data

In [1]:
from datetime import datetime
start = datetime.now()

import rasterio
import dask.array as da
import numpy as np
import pandas as pd

In [2]:
filename = './data/small.tif'
src = rasterio.open(filename)

band_names = ['coastal_blue',
              'blue',
              'green_i',
              'green',
              'yellow',
              'red',
              'red_edge',
              'nir']

band_numbers = ['1', '2', '3', '4', '5', '6', '7', '8']
pc_names = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8']

n_bands = src.meta['count']
img_shape = src.shape

In [3]:
# Read the grid values into dask/numpy arrays
bnd_list = []

for i in range(n_bands):
    bnd = da.from_array(src.read(i+1))
    bnd_list.append(bnd)

dask_band_stack = da.dstack(bnd_list)
dask_band_stack = dask_band_stack.rechunk('auto')
src_sr = dask_band_stack.compute()

In [4]:
# # Rescaling (min-max normalization)
flat_bnds = np.zeros((src_sr[:, :, 0].size, n_bands))

for i in range(n_bands):
    bnd_i = src_sr[:, :, i].flatten()
    bnd_norm = 1024*((bnd_i - bnd_i.min())/(bnd_i.max() - bnd_i.min()))
    flat_bnds[:, i] = bnd_norm

In [5]:
cor_lists = np.corrcoef(flat_bnds.transpose())

In [6]:
EigVal_cor, EigVec_cor = np.linalg.eig(cor_lists)

In [7]:

std_eigVec_table = pd.DataFrame(
    EigVec_cor,
    columns=band_numbers,
    index=band_numbers
)

std_eigVec_table = round(std_eigVec_table, 3)

In [8]:
order2 = EigVal_cor.argsort()[::-1]
EigVal_cor = EigVal_cor[order2]

In [9]:
pc_values_cor = (EigVal_cor/sum(EigVal_cor)*100).round(2)

In [10]:
## Create the PCA components for the Imagery
PC_std = np.matmul(flat_bnds, EigVec_cor)  # cross product

In [11]:
# Rescaling (min-max normalization)
PC_nrm = np.zeros(PC_std.shape)

for i in range(n_bands):
    PC_i = PC_std[:, i]
    pc_norm_mm = 1024*(PC_i - PC_i.min())/(PC_i.max() - PC_i.min())
    PC_nrm[:, i] = pc_norm_mm


In [12]:
# Reshaping
PC_std_2d_norm = np.zeros((img_shape[0], img_shape[1], n_bands))

for i in range(n_bands):
    PC_std_2d_norm[:, :, i] = PC_nrm[:, i].reshape(-1, img_shape[1])

In [13]:
# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count=8)

In [14]:
# band, row, col
tmp_std_norm = np.moveaxis(PC_std_2d_norm, [0, 1, 2], [2, 1, 0])
PC_img_std_norm = np.swapaxes(tmp_std_norm, 1, 2)

In [15]:
# Write band calculations to a new raster file
with rasterio.open('./data/analytic/0_derived/PC_img_std_norm-LRG.tif', 'w', **kwargs) as dst:
    dst.write(PC_img_std_norm.astype(rasterio.float32))

In [16]:
end = datetime.now()
diff = end-start
diff.total_seconds()


98.306884